In [2]:
import requests as rq
from bs4 import BeautifulSoup
from tqdm import tqdm

In [3]:
from dataclasses import dataclass


# 道具
@dataclass
class Property:
    name: str                       # 道具名字
    propType: str                   # 道具类型
    propQuality: str                # 道具品质
    picHref: str = None             # 图片链接(可能有未上传的图片)
    propSources: list = None        # 道具来源(后续根据表格添加)

# 人物
@dataclass
class Character:
    name: str               # 人物姓名
    picHref: str            # 图片链接
    star: int               # 人物星级
    inspiration: str        # 人物灵感
    damageType: str         # 创伤类型
    

In [29]:
# 利用单线程爬取道具信息
def getPropsInfoWithSingleThread(url):
    t = rq.get(url).text

    soup = BeautifulSoup(t, "html.parser")

    divs = list(soup.find("div", class_="resp-tab-content", style="display: block"))


    props = []
    for div in tqdm(divs):
        a = div.findNext("a")

        href = "https://wiki.biligame.com" + a.get("href")
        name = a.get("title")
        img = a.find("img")
        # 跳过部分没有图片的道具
        if not img:
            continue
        picHref = img.get("src")

        p = rq.get(href)
        s = BeautifulSoup(p.text, "html.parser")

        trs = s.find_all("tr")

        propType = trs[3].findNext("td", style="width:30%").text.replace("\n", "")
        propQuality = trs[4].findNext("td", colspan="3").text.replace("\n", "")
        # propSource = trs[7].findNext("td", colspan="3").text.replace("\n", "")

        prop = Property(name=name, picHref=picHref,
                        propQuality=propQuality,
                        # propSources=propSource,
                        propType=propType)
        props.append(prop)
        # print(prop)
        # break
    return props


url = 'https://wiki.biligame.com/reverse1999/%E9%81%93%E5%85%B7%E5%9B%BE%E9%89%B4'
props = getPropsInfoWithSingleThread(url=url)

100%|██████████| 140/140 [00:26<00:00,  5.33it/s]


In [34]:
# 利用多线程获取图鉴数据
from queue import Queue
import concurrent.futures
from tqdm import tqdm


# 爬取图鉴链接
def getPropsHref(url):
    t = rq.get(url).text
    soup = BeautifulSoup(t, "html.parser")
    divs = list(soup.find("div", class_="resp-tab-content", style="display: block"))

    hrefQueue = Queue()
    for div in divs:
        a = div.findNext("a")
        href = "https://wiki.biligame.com" + a.get("href")
        hrefQueue.put(href)
    return hrefQueue
    

# 根据图鉴链接爬取数据
def getPropsInfoWithMultiThreading(url):
    response = rq.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    img = soup.find_all("img", class_="dj-img")[0].get("src")  # 图片链接

    trs = soup.find_all("tr")

    propName = trs[2].findNext("td", style="width:30%").text.replace("\n", "")
    propType = trs[3].findNext("td", style="width:30%").text.replace("\n", "")
    propQuality = trs[4].findNext("td", colspan="3").text.replace("\n", "")

    prop = Property(name=propName, picHref=img,
                    propType=propType,
                    propQuality=propQuality
    )
    return prop


# 定义线程任务
def worker(url):
    # 调用 getPropsInfo 函数获取 prop 信息
    prop = getPropsInfoWithMultiThreading(url)
    # 将 prop 存储到列表 props 中
    props.append(prop)


url = 'https://wiki.biligame.com/reverse1999/%E9%81%93%E5%85%B7%E5%9B%BE%E9%89%B4'
hrefQueue = getPropsHref(url=url)

# 创建线程池
num_threads = 5  # 设置线程数量
props = []  # 存储 prop 的列表

# 设置进度条
total_tasks = hrefQueue.qsize()
with tqdm(total=total_tasks, desc='Progress') as pbar:
    # 创建线程池
    with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
        # 提交任务给线程池
        futures = [executor.submit(worker, hrefQueue.get()) for _ in range(total_tasks)]
        
        # 处理任务结果
        for future in concurrent.futures.as_completed(futures):
            # 更新进度条
            pbar.update(1)

# 所有任务已完成
print("All tasks completed")


Progress: 100%|██████████| 140/140 [00:11<00:00, 12.10it/s]

All tasks completed


In [37]:
# 按照输入类型保存图片到本地文件夹pics
import os


def downloadPics(rootPath, infos, type):
    if not os.path.exists(rootPath):
        os.mkdir(rootPath)

    if type == '道具':
        for info in tqdm(infos):
            if not os.path.exists(f'{rootPath}/{type}信息/'):
                os.mkdir(f'{rootPath}/{type}信息/')
            if not os.path.exists(f'{rootPath}/{type}信息/{info.propType}'):
                os.mkdir(f'{rootPath}/{type}信息/{info.propType}')
            with open(f'{rootPath}/{type}信息/{info.propType}/{info.name}.png', 'wb') as fp:
                fp.write(rq.get(info.picHref).content)
    elif type == '人物':
        for info in tqdm(infos):
            if not os.path.exists(f'{rootPath}/{type}信息/'):
                os.mkdir(f'{rootPath}/{type}信息/')
            with open(f'{rootPath}/{type}信息/{info.name}.png', 'wb') as fp:
                fp.write(rq.get(info.picHref).content)


rootPath = './pics'
downloadPics(rootPath=rootPath, infos=props, type='道具')

100%|██████████| 238/238 [00:32<00:00,  7.37it/s]


In [19]:
# 表格初始化
import traceback

import openpyxl
from openpyxl.styles import PatternFill
from openpyxl.drawing.image import Image
from openpyxl.utils import get_column_letter
from openpyxl.styles import Alignment
from openpyxl.styles import Border, Side


# 在表格前三行输入道具信息
def inputPropsInfo(sheet, prop, color, column):
    # 输入道具名称
    sheet[get_column_letter(column) + '1'] = prop.name
    sheet[get_column_letter(column)  + '1'].fill = color

    # 放置道具图片 (重复放置时，新图片会重叠在旧图片上，删除时需要多次删除)
    img = Image('./pics/道具信息/养成材料/' + prop.name + '.png')
    sheet.add_image(img, get_column_letter(column) + '2')
    sheet[get_column_letter(column)  + '2'].fill = color

    # 输入道具品质
    sheet[get_column_letter(column)  + '3'] = prop.propQuality
    sheet[get_column_letter(column)  + '3'].fill = color


# 设置文字居中
def textCenter(sheet):
    alignment = Alignment(horizontal='center', vertical='center')
    for row in sheet.iter_rows():
        for cell in row:
            cell.alignment = alignment


# 修改单元格大小
def modifyCellSize(sheet, width, height):
    # 图片放置在第二行，从第三列开始往后
    for i in range(len(props)): 
        #  从第三列 'C' 开始调整
        column_letter = get_column_letter(i+3)
        sheet.column_dimensions[column_letter].width = width
    # 调整第二行高度
    sheet.row_dimensions[2].height = height


# 初始化前三行内容
def initFirstThreeLines(sheet, props):
    # 单元格颜色
    fill_dict = {
        '绿':  PatternFill(patternType='solid',fgColor='00FF00'),
        '蓝':  PatternFill(patternType='solid',fgColor='66CCFF'),
        '紫':  PatternFill(patternType='solid',fgColor='FF00FF'),
        '黄':  PatternFill(patternType='solid',fgColor='FFFF00'),
        '金':  PatternFill(patternType='solid',fgColor='FF6633'),
    }
    # 按照品质进行排序
    qualityOrder = ['绿', '蓝', '紫', '黄', '金']
    props.sort(key=lambda x: qualityOrder.index(x.propQuality))

    # i表示 表格第i列，从 第3列 开始
    for i in range(len(props)):
        inputPropsInfo(sheet=sheet, prop=props[i], color=fill_dict[props[i].propQuality], column=i+3)


# 在表格前两列输入关卡信息，包括 关卡名称 和 关卡难度（故事/厄险）
def inputLevelInfo(sheet):
    # 存储关卡数据，键表示 第key章，值表示 第value关
    level = {
        1: [1, 4, 5, 6 ,7, 8, 11, 12, 14, 15, 16],       # 11
        2: [1, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14],    # 13
        3: [3, 5, 6, 7, 8, 9, 11, 13, 15],               # 9
        4: [3, 4, 5, 6, 10, 11, 13, 16, 18, 20, 21]      # 11
    }

    # 设置边框样式
    borderStyle = Side(style='thin')  # 边框样式为细线
    borderTop = Border(top=borderStyle)  # 仅有上边框
    borderBottom = Border(bottom=borderStyle)  # 仅有下边框 
    column_count = sheet.max_column  # 列数

    # i, j 表示 第i章第j关，count表示行数，从 第4行 开始
    count = 4
    for i in level:
        for j in level[i]:
            sheet['A' + f'{count}'] = f'{i}-{j}'
            sheet['B' + f'{count}'] = '故事'
            # 设置边框样式
            for column in range(1, column_count + 1):
                cell = sheet.cell(row=count, column=column)
                cell.border = borderTop
            count += 1

            sheet['A' + f'{count}'] = None
            sheet['B' + f'{count}'] = '厄险'
            # 设置边框样式
            for column in range(1, column_count + 1):
                cell = sheet.cell(row=count, column=column)
                cell.border = borderBottom
            count += 1


def initExcel(path, props):
    if os.path.exists(path):
        workbook = openpyxl.load_workbook(path)
    else:
        workbook = openpyxl.Workbook()
    sheet = workbook.active

    props = [prop for prop in props if prop.propType == '养成材料']

    try:
        # 调整第二列单元格大小，来放置图片
        modifyCellSize(sheet=sheet, width=15, height=82.2)

        # 初始化前三行内容，包括 道具名称、道具图片和道具品质，并设置单元格背景颜色
        initFirstThreeLines(sheet=sheet, props=props)

        # 初始化前两列内容，包括 关卡名称 和 难度(故事/厄险)
        inputLevelInfo(sheet=sheet)

        # 设置冻结窗口
        # 设置要冻结的行和列的索引
        freeze_row = 4  # 冻结的行索引（从4开始）
        freeze_column = 3  # 冻结的列索引（从3开始）
        sheet.freeze_panes = sheet.cell(row=freeze_row, column=freeze_column)

        # 设置文字居中
        textCenter(sheet=sheet)

        # 保存文件
        workbook.save(path)
        
    except Exception as e:
        workbook.save(path)
        print(f"An error occurred: {str(e)}")
        traceback.print_exc()


initExcel(path='./材料.xlsx', props=props)

In [19]:
# 修改excel中字体颜色
from openpyxl.styles import Font


workbook = openpyxl.load_workbook('./材料.xlsx')
sheet = workbook.active

red = Font(name="微软雅黑", color="FF0000")
blue = Font(name="微软雅黑", color="66CCFF")
green = Font(name="微软雅黑", color="00FF00")
purple = Font(name="微软雅黑", color="FF00FF")
black = Font(name="微软雅黑", color="000000")
yellow = Font(name="微软雅黑", color="FFFF00")
orange = Font(name="微软雅黑", color="FF6633")

for row in sheet.iter_rows():
    for cell in row:
        if cell.value == "极小概率":
            cell.font = green
        elif cell.value == "概率":
            cell.font = blue
        elif cell.value == "大概率":
            cell.font = purple
        elif cell.value == "固定":
            cell.font = orange


workbook.save('./材料.xlsx')